In [1]:
import requests
import re
import time
import pandas as pd
from lxml import etree
from urllib.request import urlopen, Request

In [2]:
url = "http://www.twbest1.com/"

def getPage(url):
    res = requests.get(url)
    res.encoding = 'big5'
    htmltext = res.text
    return htmltext

def gethreflist(htmltext):
    xdoc = etree.HTML(htmltext)
    href_list = list(xdoc.xpath('//area//@href'))
    href_list = list(filter(lambda x:re.search(r"(https://www\.twbest1.com)/pc[0-9]*/page[0-9]*\.htm*",x),href_list))
    return href_list

# replace html reserve sign to normal sign
def reserve_trans(htmltext):
    htmltext = re.sub(r"\u0020","",htmltext)
    htmltext = re.sub(r"\\n","",htmltext)
    htmltext = re.sub(r"&lt;",r"<",htmltext)
    htmltext = re.sub(r"&nbsp;",r"<",htmltext)
    htmltext = re.sub(r"&gt;",r">",htmltext)
    htmltext = re.sub(r"&amp;",r"&",htmltext)
    htmltext = re.sub(r"&quot;",r'"',htmltext)
    htmltext = re.sub(r"&apos;",r"'",htmltext)
    return htmltext

def parser_simple(htmltext):
    xdoc = etree.HTML(htmltext)
    site_name = xdoc.xpath("//title/text()")
    area = re.findall(r"(..\u5e02)(..\u5340)|(..\u7e23)(..\u9109)|(..\u7e23)(..\u93ae)|(..\u7e23)(..\u5e02)",htmltext)
    return list(zip(site_name,area))

In [9]:
url1 = "https://www.twbest1.com/pc001048/page070.htm"
url2 = "https://www.twbest1.com/pc100150/page237.htm"
print(parser_simple(reserve_trans(getPage(url1))))
print(parser_simple(reserve_trans(getPage(url2))))

[('菁桐老街', ('新北市', '平溪區', '', '', '', '', '', ''))]
[('原住民文化主題公園&忘憂湖', ('台北市', '士林區', '', '', '', '', '', ''))]


In [3]:
href_list = gethreflist(getPage(url))
data = [parser_simple(reserve_trans(getPage(href))) for href in href_list]

In [4]:
data

[[('清水斷崖', ('', '', '花蓮縣', '秀林鄉', '', '', '', ''))],
 [('太平山國家森林遊樂區', ('', '', '宜蘭縣', '大同鄉', '', '', '', ''))],
 [('桃園機場捷運(沿線景點)', ('桃園市', '蘆竹區', '', '', '', '', '', ''))],
 [],
 [],
 [],
 [('基隆正濱漁港', ('基隆市', '中正區', '', '', '', '', '', ''))],
 [('阿根納造船廠遺址', ('基隆市', '中正區', '', '', '', '', '', ''))],
 [('基隆碧砂漁港', ('基隆市', '中正區', '', '', '', '', '', ''))],
 [('基隆和平島', ('基隆市', '中正區', '', '', '', '', '', ''))],
 [('望幽谷', ('基隆市', '中正區', '', '', '', '', '', ''))],
 [('海門天險', ('基隆市', '中正區', '', '', '', '', '', ''))],
 [],
 [],
 [('宜蘭．梅花湖', ('', '', '宜蘭縣', '冬山鄉', '', '', '', ''))],
 [('宜蘭．福山植物園', ('', '', '宜蘭縣', '員山鄉', '', '', '', ''))],
 [('宜蘭明池', ('', '', '宜蘭縣', '大同鄉', '', '', '', ''))],
 [('宜蘭望龍埤', ('', '', '宜蘭縣', '員山鄉', '', '', '', ''))],
 [('南方澳漁港', ('', '', '', '', '宜蘭縣', '蘇澳鎮', '', ''))],
 [],
 [],
 [],
 [('幾米廣場&丟丟噹森林', ('', '', '', '', '', '', '宜蘭縣', '宜蘭市'))],
 [],
 [('宜蘭•蘭陽博物館', ('', '', '', '', '宜蘭縣', '頭城鎮', '', ''))],
 [('烏石港', ('', '', '', '', '宜蘭縣', '頭城鎮', '', ''))],
 [('冬山車站', ('',

In [5]:
temp = open("site_temp.txt","wt",encoding='utf-8')
n = temp.write(str(data))
temp.close()

In [109]:
data1 = str(data)

data1 = re.sub(r"\(|\)","",data1)

data2 = data1.replace(r"於",r"台")

data3 = data2.replace(r"之",r"竹")

data3 = eval(data3)

data3 = [list(filter(None, sub_list)) for sub_list in data3]

data3[0]

['清水斷崖', '花蓮縣', '秀林鄉']

In [110]:
df = pd.DataFrame(data3,columns=['site','city','town'])
df = df.mask(df.astype(object).eq('None')).dropna()
df

,site,city,town
0,清水斷崖,花蓮縣,秀林鄉
1,太平山國家森林遊樂區,宜蘭縣,大同鄉
2,桃園機場捷運沿線景點,桃園市,蘆竹區
6,基隆正濱漁港,基隆市,中正區
7,阿根納造船廠遺址,基隆市,中正區
...,...,...,...
356,鳳山懷舊竹旅,高雄市,鳳山區
357,東港大鵬灣,屏東縣,東港鎮
358,屏東東港~東隆宮,屏東縣,東港鎮
359,萬巒鄉情&豬腳美食,屏東縣,萬巒鄉


In [117]:
city_and_town = df.groupby(['city','town'])
city_and_town_df = city_and_town.size().to_frame()
city_and_town_df.reset_index(inplace=True)
city_and_town_df.rename({0:'sites'},inplace=True,axis=1)

for i,(city,town) in enumerate(zip(city_and_town_df['city'],city_and_town_df['town'])):
    city_and_town_df.loc[i,'sites'] = city_and_town.get_group((city,town))['site'].tolist()
    
city_and_town_df.loc[0,'sites'] = [city_and_town_df.loc[0,'sites']]

In [118]:
city_and_town_df

,city,town,sites
0,南投縣,仁愛鄉,[清境農場]
1,南投縣,南投市,[南投中興新村]
2,南投縣,竹山鎮,"[杉林溪, 竹山紫南宮]"
3,南投縣,集集鎮,[集集車站]
4,南投縣,鹿谷鄉,"[溪頭風光, 溪頭妖怪村]"
...,...,...,...
127,高雄市,美濃區,[美濃風情]
128,高雄市,鳥松區,[高雄澄清湖]
129,高雄市,鳳山區,[鳳山懷舊竹旅]
130,高雄市,鹽埕區,"[真愛碼頭, 駁二藝術特區]"


In [68]:
city_and_town_df['city'].unique()

array(['南投縣', '台中市', '台北市', '台南市', '台東縣', '嘉義縣', '基隆市', '宜蘭縣', '屏東縣',
       '彰化縣', '新北市', '新竹市', '新竹縣', '桃園市', '桃園縣', '臺北市', '花蓮縣', '苗栗縣',
       '雲林縣', '高雄市'], dtype=object)

In [99]:
for town in city_and_town_df['town']:
    print(town)

仁愛鄉
南投市
竹山鎮
集集鎮
鹿谷鄉
南屯區
和平區
大甲區
新社區
清水區
石岡區
中山區
中正區
信義區
內湖區
北投區
士林區
大同區
大安區
文山區
松山區
萬華區
七股區
中西區
仁德區
北門區
安南區
安平區
後壁區
白河區
台東市
成功鎮
池上鄉
關山鎮
布袋鎮
新港鄉
東石鄉
竹崎鄉
中正區
三星鄉
五結鄉
冬山鄉
南澳鄉
員山鄉
大同鄉
宜蘭市
礁溪鄉
羅東鎮
蘇澳鎮
頭城鎮
恆春鎮
東港鎮
萬巒鄉
彰化市
溪湖鎮
福興鄉
芳苑鄉
鹿港鎮
三峽區
三芝區
三重區
中和區
八里區
土城區
坪林區
平溪區
新店區
板橋區
林口區
永和區
汐止區
淡水區
深坑區
烏來區
瑞芳區
石碇區
石門區
萬里區
貢寮區
金山區
鶯歌區
之北區
五峰鄉
北埔鄉
峨眉鄉
新埔鎮
橫山鄉
竹北市
竹東鎮
關西鎮
中壢區
大園區
大溪區
復興區
桃園區
蘆竹區
龍潭區
大溪鎮
復興鄉
龍潭鄉
中正區
光復鄉
吉安鄉
壽豐鄉
富里鄉
秀林鄉
花蓮市
三義鄉
南庄鄉
大湖鄉
後龍鎮
泰安鄉
獅潭鄉
苗栗市
銅鑼鄉
頭屋鄉
北港鎮
口湖鄉
古坑鄉
斗六市
斗南鎮
西螺鎮
三民區
大樹區
左營區
旗津區
田寮區
美濃區
鳥松區
鳳山區
鹽埕區
鼓山區


In [121]:
s = str()

for town,site in zip(city_and_town_df['town'],city_and_town_df['sites']):
    
    s += f"""
    ### {town}
    """
    for sub_site in site:
        s += f"""
    - [ ] {sub_site}
        """

In [106]:
print(city_and_town_df['city'].unique()[0])
print(city_and_town_df['city'].unique()[-1])

南投縣
高雄市


In [55]:
city_and_town_df.to_csv("taiwansite.csv",encoding="utf-8",index=False)

In [122]:
print(s)


    ### 仁愛鄉
    
    - [ ] 清境農場
        
    ### 南投市
    
    - [ ] 南投中興新村
        
    ### 竹山鎮
    
    - [ ] 杉林溪
        
    - [ ] 竹山紫南宮
        
    ### 集集鎮
    
    - [ ] 集集車站
        
    ### 鹿谷鄉
    
    - [ ] 溪頭風光
        
    - [ ] 溪頭妖怪村
        
    ### 南屯區
    
    - [ ] 台中彩虹眷村
        
    ### 和平區
    
    - [ ] 武陵農場
        
    ### 大甲區
    
    - [ ] 大甲鎮瀾宮
        
    ### 新社區
    
    - [ ] 新社悠遊
        
    ### 清水區
    
    - [ ] 清水荷花古厝
        
    - [ ] 高美濕地
        
    ### 石岡區
    
    - [ ] 石岡風情
        
    ### 中山區
    
    - [ ] 大佳河濱公園
        
    - [ ] 台北忠烈祠
        
    - [ ] 劍潭古寺
        
    - [ ] 行天宮
        
    ### 中正區
    
    - [ ] 中正紀念堂
        
    - [ ] 台北中山堂
        
    - [ ] 撫臺街洋樓
        
    - [ ] 台北市．自來水博物館
        
    - [ ] 土銀展示館
        
    - [ ] 國立台灣博物館
        
    - [ ] 郵政博物館
        
    - [ ] 台北植物園
        
    - [ ] 台北古城門
        
    - [ ] 臺北賓館
        
    - [ ] 228和平公園
        
    ### 信義區
    
    - [ ] 象山自然步道
        
    - [ ] 